In [48]:
import torch
import os

In [49]:

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '2'


In [50]:
from natsort import natsorted
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

all_blocks = []
all_labels = []

all_files_path = natsorted(glob('merged_data_pkl/*'))

for path in tqdm(all_files_path):
    with open(path, 'rb') as file:
        data_dict = pickle.load(file)
        all_blocks.append(data_dict['data'])
        all_labels.append(data_dict['label'])

X = np.array(all_blocks)
Y = np.array(all_labels)

print(X.shape, Y.shape)


 50%|████▉     | 4190/8400 [00:00<00:00, 20810.52it/s]

100%|██████████| 8400/8400 [00:00<00:00, 19775.80it/s]


(8400, 32, 250) (8400,)


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms


In [52]:
class SoundCNN(nn.Module):
    def __init__(self):
        super(SoundCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 4 * 31, 512)  # Corrected dimension
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)  # Assuming 10 different classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [53]:
# Reshape data to include channel dimension (1, 32, 250)
X = X.reshape(X.shape[0], 1, 32, 250)

Y = Y.astype(np.int64)


# Convert to torch tensors
tensor_X = torch.Tensor(X)  # transform to torch tensor
tensor_Y = torch.Tensor(Y)

# Create your dataloader
my_dataset = TensorDataset(tensor_X, tensor_Y)
train_loader = DataLoader(my_dataset, batch_size=64, shuffle=True)


In [54]:
model = SoundCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [55]:
from torch.utils.data import random_split

# Assuming tensor_X and tensor_Y are already defined
dataset = TensorDataset(tensor_X, tensor_Y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [56]:
def validation_accuracy(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()  # Set the model back to train mode
    return 100 * correct / total


In [57]:
tensor_X = torch.Tensor(X)  # Assuming X is your input data.
tensor_Y = torch.Tensor(Y).long()  # Convert labels to long

# Create dataset
my_dataset = TensorDataset(tensor_X, tensor_Y)
train_size = int(0.8 * len(my_dataset))
val_size = len(my_dataset) - train_size

train_dataset, val_dataset = random_split(my_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [58]:
import os

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, save_path):
    model.train()
    best_accuracy = 0.0  # To track the best model

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_acc = validation_accuracy(model, val_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_acc}%')
        
        # Save the best model if the current model is better
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': running_loss,
                'accuracy': val_acc
            }, save_path)
            print(f'Saved better model at epoch {epoch+1} with validation accuracy: {val_acc}%.')

# Define model, criterion, optimizer, and file path for saving the model
model = SoundCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
save_path = './SoundCNN_model.pth'  # Modify path as needed

# Call training function
train_model(model, train_loader, val_loader, criterion, optimizer, 18, save_path)


Epoch 1, Loss: 2.507578534171695, Validation Accuracy: 14.166666666666666%
Saved better model at epoch 1 with validation accuracy: 14.166666666666666%.
Epoch 2, Loss: 2.1743020103091286, Validation Accuracy: 15.297619047619047%
Saved better model at epoch 2 with validation accuracy: 15.297619047619047%.
Epoch 3, Loss: 2.1548023927779423, Validation Accuracy: 14.107142857142858%
Epoch 4, Loss: 2.1738751888275147, Validation Accuracy: 18.392857142857142%
Saved better model at epoch 4 with validation accuracy: 18.392857142857142%.
Epoch 5, Loss: 2.1567490941002254, Validation Accuracy: 16.785714285714285%
Epoch 6, Loss: 2.1324926580701558, Validation Accuracy: 19.166666666666668%
Saved better model at epoch 6 with validation accuracy: 19.166666666666668%.
Epoch 7, Loss: 2.141413727260771, Validation Accuracy: 14.94047619047619%
Epoch 8, Loss: 2.2157293910071965, Validation Accuracy: 15.833333333333334%
Epoch 9, Loss: 2.1550103051321847, Validation Accuracy: 15.892857142857142%
Epoch 10, L

In [59]:
test_data = pd.read_excel('/home/project/new/Test.xlsx')

In [60]:
import numpy as np
import pandas as pd
import torch

# Assuming `test_data` is already loaded and is a Pandas DataFrame
all_test_blocks = []
block_height = 32
num_columns_in_test = 750  # Columns per test block

# Iterate over the dataset in chunks of 32 rows
for start_row in range(0, test_data.shape[0], block_height):
    if start_row + block_height > test_data.shape[0]:
        continue  # Skip incomplete blocks
    chunk = test_data.iloc[start_row:start_row + block_height]

    # Split each 750-column block into three 250-column blocks
    for i in range(3):  # Since 750/250 = 3
        start_col = i * 250
        end_col = start_col + 250
        block = chunk.iloc[:, start_col:end_col]
        all_test_blocks.append(block.values.reshape(1, block_height, 250))  # Adding channel dimension

# Convert blocks to PyTorch tensors
X_test_blocks = torch.tensor(all_test_blocks, dtype=torch.float)


In [61]:
# Assuming the model is already defined as `SoundCNN`
model = SoundCNN()
model.load_state_dict(torch.load('/home/project/new/SoundCNN_model.pth')['model_state_dict'])

# Transfer model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode


SoundCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=7936, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [62]:
X_test_blocks = X_test_blocks.to(device)  # Move the test data to the same device as the model
with torch.no_grad():  # Ensure no gradients are computed during inference
    predictions = model(X_test_blocks)
    predicted_classes = torch.argmax(predictions, dim=1).cpu().numpy()  # Move predictions back to CPU and convert to numpy


In [63]:
import pandas as pd

# `predicted_classes` is the numpy array containing the class indices predicted by the model
predictions_df = pd.DataFrame(predicted_classes, columns=['PredictedClass'])


In [64]:
# Specify the path and name of the CSV file
csv_file_path = './predictions.csv'

# Save the DataFrame to a CSV file without the index column
predictions_df.to_csv(csv_file_path, index=False)


In [65]:
label_map = {
    0: "Tiger",
    1: "Snake",
    2: "Wolf",
    3: "Bear",
    4: "Rabbit",
    5: "Monkey",
    6: "Eagle",
    7: "Dolphin",
    8: "Koala"
}


In [66]:
import pandas as pd

# Assuming `predicted_classes` is already available as a numpy array
predictions_df = pd.DataFrame(predicted_classes, columns=['PredictedClass'])

# Replace numeric labels with animal names using the mapping
predictions_df['PredictedClass'] = predictions_df['PredictedClass'].replace(label_map)


In [67]:
# Specify the path and name of the CSV file
csv_file_path = './animal_predictions.csv'

# Save the DataFrame to a CSV file without the index column
predictions_df.to_csv(csv_file_path, index=False)


In [68]:
import pandas as pd
import numpy as np
from collections import Counter
import random

# Load predictions
predictions_df = pd.read_csv('./animal_predictions.csv')

# Function to determine the majority or random in case of a tie
def majority_or_random(labels):
    count = Counter(labels)
    max_freq = max(count.values())
    candidates = [label for label, freq in count.items() if freq == max_freq]
    return random.choice(candidates)

# Group predictions in groups of 3 and apply the function
grouped_labels = []
for i in range(0, len(predictions_df), 3):
    labels = predictions_df['PredictedClass'][i:i+3].tolist()
    if len(labels) == 3:  # Ensure it's a full group of 3
        grouped_label = majority_or_random(labels)
        grouped_labels.append(grouped_label)

# Create new DataFrame
final_predictions_df = pd.DataFrame(grouped_labels, columns=['MajorityVotedClass'])


In [69]:
# Save the DataFrame to a CSV file
final_csv_file_path = './final_animal_predictions.csv'
final_predictions_df.to_csv(final_csv_file_path, index=False)


In [70]:
import pandas as pd

# Load the CSV file
file_path = './final_animal_predictions.csv'
predictions_df = pd.read_csv(file_path)


In [71]:
# Generate the 'ID' column
predictions_df['ID'] = ['id_' + str(index + 1) for index in predictions_df.index]


In [72]:
# Reorder columns to make 'ID' the first column
column_order = ['ID', 'MajorityVotedClass']
predictions_df = predictions_df[column_order]


In [73]:
# Save the updated DataFrame to a new CSV file
updated_csv_file_path = './updated_final_animal_predictions.csv'
predictions_df.to_csv(updated_csv_file_path, index=False)
